In [39]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import statsmodels.api as sm
import category_encoders as ce


# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss
from sklearn.impute import KNNImputer

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('once')

In [40]:
train_set = pd.read_csv('train_prepared.csv', encoding='utf-8')
test_set = pd.read_csv('test_prepared.csv', encoding='utf-8')

train_set = train_set.replace([-np.inf, np.inf], np.nan)
test_set = test_set.replace([-np.inf, np.inf], np.nan)

In [22]:
X = train_set.drop(columns=['Target', 'Opportunity_ID'])
y = train_set.Target

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [24]:
'''
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 2000, num = 5)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
'''

"\nn_estimators = [int(x) for x in np.linspace(start = 50, stop = 2000, num = 5)]\n\nmax_features = ['auto', 'sqrt']\n\nmax_depth = [int(x) for x in np.linspace(10, 110, num = 5)]\nmax_depth.append(None)\n\nmin_samples_split = [2, 5, 10]\n\nmin_samples_leaf = [1, 2, 4]\n\nbootstrap = [True, False]\n\nrandom_grid = {'n_estimators': n_estimators,\n               'max_features': max_features,\n               'max_depth': max_depth,\n               'min_samples_split': min_samples_split,\n               'min_samples_leaf': min_samples_leaf,\n               'bootstrap': bootstrap}\nprint(random_grid)\n"

In [25]:
'''
clf=RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)

rf_random.fit(X_train,y_train)

y_pred=rf_random.predict_proba(X_test)
'''

'\nclf=RandomForestClassifier()\nrf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 100, cv = 2, verbose=2, random_state=42, n_jobs = -1)\n\nrf_random.fit(X_train,y_train)\n\ny_pred=rf_random.predict_proba(X_test)\n'

In [26]:
#rf_random.best_params_

In [27]:
#print(log_loss(y_test, y_pred))

In [29]:
c = RandomForestClassifier(n_estimators= 50, min_samples_split= 2, 
                           min_samples_leaf=5, max_features= 'sqrt', 
                           max_depth= 25,bootstrap= False)

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer

numeric_cols = X_train.select_dtypes(include=['int', 'float64']).columns.to_list()
#numeric_cols = ['Total_Amount_Sum_USD', 'Total_Taxable_Amount_USD']

# Transformaciones para las variables numéricas
numeric_transformer = Pipeline(
                        steps=[
                            ('imputer', SimpleImputer()),
                            ('scaler', Normalizer())
                        ]
                      )

preprocessor = ColumnTransformer(
                    transformers=[
                        ('numeric', numeric_transformer, numeric_cols)
                        ],
                    remainder='passthrough'
                    )

pipe = Pipeline([('preprocessing', preprocessor),('rf',c)])

pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   Normalizer())]),
                                                  ['Convertibility',
                                                   'Total_Amount_Sum_USD',
                                                   'Total_Taxable_Amount_USD',
                                                   'Year_Creation',
                                                   'Month_Creation',
                                                   'Year_Delivery',
                                                   'Month_Delivery',
                        

In [30]:
from sklearn.metrics import f1_score

print(log_loss(y_test, pipe.predict_proba(X_test)))
f1_score(y_test, pipe.predict(X_test))

0.38027978628876286


0.8471490177287972

In [31]:
print(log_loss(y_train, pipe.predict_proba(X_train)))
f1_score(y_train, pipe.predict(X_train))

0.14935675693675415


0.9815159409502543

# EN TEST_SET

In [32]:
test_pred = pipe.predict_proba(test_set.drop(columns=['Opportunity_ID', 'Target']))

In [33]:
test_set['Prediction'] = pd.DataFrame(test_pred)[1].to_list()
test_set.loc[:, ['Opportunity_ID', 'Prediction']]

,Opportunity_ID,Prediction
0,10689,0.879619
1,10690,0.563016
2,10691,0.677508
3,10692,0.231444
4,10693,0.916643
...,...,...
1562,12364,0.984000
1563,12365,0.645929
1564,12366,0.611381
1565,12367,0.342802


In [34]:
test_set['Prediction_Aux'] = 1 - test_set.Prediction
test_set.loc[:, ['Opportunity_ID', 'Prediction', 'Prediction_Aux']]

,Opportunity_ID,Prediction,Prediction_Aux
0,10689,0.879619,0.120381
1,10690,0.563016,0.436984
2,10691,0.677508,0.322492
3,10692,0.231444,0.768556
4,10693,0.916643,0.083357
...,...,...,...
1562,12364,0.984000,0.016000
1563,12365,0.645929,0.354071
1564,12366,0.611381,0.388619
1565,12367,0.342802,0.657198


In [35]:
log_loss(test_set.Target, test_set.Prediction_Aux)

1.945666504660557

In [36]:
log_loss(test_set.Target, test_set.Prediction)

0.4710638588746126

In [17]:
submission = test_set.loc[:, ['Opportunity_ID', 'Prediction']]
submission.columns = ['Opportunity_ID', 'Target']

In [18]:
submission.head()

,Opportunity_ID,Target
0,10689,0.839667
1,10690,0.563556
2,10691,0.424024
3,10692,0.400992
4,10693,0.923929


In [19]:
submission.to_csv('predictions/sub_rf.csv', index=False)